## Install packages & dependecies:

In [ ]:
"""
%%bash
#Cyrille's github
git clone https://github.com/cdelley/scyBCB
pip install git+https://github.com/cdelley/scyBCB

#Install python dependencies
pip install watermark
pip install anndata
pip install "numpy<2"
pip install xopen
pip install PyQt5

#Install GATK 4.1.3.0 because Java on cluster is 11 and only compatible with GATK 4.1.x.x.
!unzip /home/caleb/scDNA_analysis/gatk-4.1.3.0.zip

#Install bwa-mem2 (Linux)
!curl -L https://github.com/bwa-mem2/bwa-mem2/releases/download/v2.2.1/bwa-mem2-2.2.1_x64-linux.tar.bz2 \
  | tar jxf -
  
#Install yaml
!pip install pyyaml
"""

## Import libraries & Tool setup:

In [ ]:
# This is the github repo
import scyBCB.CyGeno as dabm
import scyBCB.CyBCB as bcb

import os
import json
import time
import subprocess
import gzip
import re
import numpy as np
import pandas as pd
import matplotlib
import subprocess
import yaml

#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

%load_ext watermark
%watermark -v -iv

from collections import Counter
from multiprocessing.pool import ThreadPool
from multiprocessing import Process

In [ ]:
# Load YAML configuration file
with open('config_example.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
config

In [ ]:
!chmod +x {config['demuxbyname_path']}

## File/Folder directories (change here):

In [ ]:
# Barcode whitelist help to calculate cell barcode base offset (1-3 nucleotide btw BC1 and BC2) if beads are synthesized in split-pool approaches.
barcodes = [config['barcode_whitelist_1'],
            config['barcode_whitelist_2']]
bc_dicts = []
for bc_json in barcodes:
    with open(bc_json, 'r') as fin:
        bc_dicts.append(json.load(fin))
        
# Forward and reverse primer .csv files of the targeted DNA panels (AML or ML):
primers = pd.read_csv(config['primer_csv'])
primer_dict_fw = {s[:19]:n+'_fw' for (s,n) in zip(list(primers['fwd_seq']), list(primers['AmpID']))}
primer_dict_rev = {s[:19]:n+'_rev' for (s,n) in zip(list(primers['rev_seq']), list(primers['AmpID']))}

# Directory to Read 1 and Read 2 files
f1 = config['read_1_file']
f2 = config['read_2_file']

# Output folder (to dump processed sequencing file in)
out_folder = config['fastq_out_folder']
os.makedirs(out_folder, exist_ok=True)

In [ ]:
# Barcode template:
# MissionBio beads
MB_seq_pos_dict = {
    'bc'  : [(0,0,9), (0,23,32)], # two barcodes
    'feature' : [(0,47,66), (1,0,19)],        # features, in this case, are the primer positions for stats, normally this is to indicate antibody barcode tags
    'seq': [(0,66,151), (1,0,151)],           # sequence positions including the primer region. These are copied to the fastq output
}

# Lab-made beads:
LM_seq_pos_dict = {
    'bc'  : [(0,0,11), (0,12,20), (0,24,32)], # three barcodes
    'feature' : [(0,51,70), (1,0,19)],        
    'seq': [(0,51,151), (1,0,151)],
}

In [ ]:
# bwa-mem2 index location
bwamem2 = config['bwamem2']
# folder directories for bam and vcf file
bam_dir = f"{out_folder}/bam/"
bam_with_RG_dir = f"{out_folder}/bam_with_RG/"
vcf_dir = f"{out_folder}/vcf/"
vcf_csv_dir = f"{out_folder}/vcf_csv/"
os.makedirs(bam_dir, exist_ok=True)
os.makedirs(bam_with_RG_dir, exist_ok=True)
os.makedirs(vcf_dir, exist_ok=True)

# reference genome file and primer location for variant callling
human_fasta_file = config['human_fastq_file']
interval_file = config['interval_file'] #only look at amplicons that contain GM1 & GM2 variants

## Filter reads based on cell barcode & fwd/rvs primer alignment:

In [ ]:
%%time
exp = dabm.sequences.write_good_reads(
    file1=f1,
    file2=f2, 
    bc_correction_dic=bc_dicts,
    primer_dic=[primer_dict_fw, primer_dict_rev],
    out_path=out_folder,
    seq_positions=MB_seq_pos_dict,
    verbose=True, 
    #down_sample=10**3
)

In [ ]:
def recover_barcode(barcode_count_dict, low_count_filter=200, batch_label=''):
    """
    This functions filter barcodes based on the total number of reads per barcode.
    The default minimum total of reads required is 200.
    barcode_count_dict: dictionaries from write_good_reads function where each key is the barcode, each value is the total read counts
    low_count_filter: minimum of reads per barcode to be selected for downstream processing
    """
    
    # count reads per barcode
    reads = np.array(list(barcode_count_dict.values()))
    _thresh_idx = reads >= low_count_filter

    bcs = np.array(list(barcode_count_dict.keys()))[_thresh_idx]  
    cell_barcodes = np.sort(bcs)

    barcode_count_touples = list(barcode_count_dict.items())
    return cell_barcodes

In [ ]:
barcode_recover = recover_barcode(exp.bc_groups)
barcode_string = barcode_recover.astype(str)
np.savetxt(out_folder + "/filtered_barcode_string.txt", barcode_string, fmt='%s')

In [ ]:
# demultiplex the R1 and R2 into multiple fastq.gz file for each barcode
with open(f'{out_folder}/barcode_recover', 'w') as fout:
    for i in barcode_recover:
        fout.write(str(i)+',')

# Change here for path to bbmap
_pp1 = f"{config['demuxbyname_path']} -Xmx60g "
_pp2 = f"in={out_folder}/R1.fastq.gz in2={out_folder}/R2.fastq.gz out={out_folder}/demuxed_all/cell_%.fastq.gz  "
_pp3 = f"delimiter=_ suffixmode=t names={out_folder}/filtered_barcode_string.txt"

cmd = _pp1+_pp2+_pp3
subprocess.run(cmd, shell = True, executable="/bin/bash")

## Target amplicon read counts:

In [ ]:
def fastq_to_txt(input_folder, output_folder):
    """
    This function is to extract R2 from .fastq.gz files into txt files
    We can cut down this function somehow in the future to increase speed of the pipeline
    input_folder: folder containing .fastq.gz files
    output_folder: folder containing .txt files (that are converted from .fastq.gz files)
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Regular expression to capture the sequence after "-R2-" up to the underscore
    pattern = re.compile(r"@.*-R2-([A-Z]+)")

    # Loop through each FASTQ file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".fastq.gz"):
            input_path = os.path.join(input_folder, filename)
            output_filename = filename.replace(".fastq.gz", "_R2_sequences.txt")
            output_path = os.path.join(output_folder, output_filename)

            # Open the output file to write extracted sequences
            with open(output_path, "w") as output_file:
                # Open and read the compressed FASTQ file
                with gzip.open(input_path, "rt") as fastq_file:
                    for line in fastq_file:
                        # Check if the line is a header with "-R2-"
                        match = pattern.match(line)
                        if match:
                            # Write the extracted sequence to the output file
                            sequence_after_R2 = match.group(1)
                            output_file.write(sequence_after_R2 + "\n")

            print(f"Processed {filename} and saved to {output_filename}")

In [ ]:
fastq_to_txt(out_folder+"/demuxed_all", out_folder+"/R2_txt")

In [ ]:
# Start counting number of reads align to each amplicon per R2 values
R2_txt_folder = out_folder + "/R2_txt"
# Initialize an empty DataFrame to store the counts
count_table = pd.DataFrame(columns=primer_dict_rev.values())

# Loop through each .txt file in the output directory
for filename in os.listdir(R2_txt_folder):
    if filename.endswith("_R2_sequences.txt"):
        file_path = os.path.join(R2_txt_folder, filename)

        # Count occurrences where a sequence aligns with any target primer
        with open(file_path, "r") as f:
            sequences = f.read().splitlines()
            file_counts = {target_name: 0 for target_name in primer_dict_rev.values()}

            for sequence in sequences:
                for target_seq, target_name in primer_dict_rev.items():
                    # Check if target sequence is a substring of the read sequence
                    if sequence in target_seq:
                        file_counts[target_name] += 1

        # Add the counts as a new row in the DataFrame, with the file name as the index
        count_table.loc[filename] = file_counts

# Display the final table
count_table.index.name = "File"
count_table.index = count_table.index.str.split('_R2').str[0]
count_table.to_csv(out_folder + "/target_amplicon_counts.csv")

## Read-count related metrics:

In [ ]:
def gini_coefficient(read_counts):
    """    
    This function calculates the Gini coefficient per barcode
    
    Parameters:
    read_counts (list or numpy array): A list or array of read counts per barcode
    
    Returns:
    float: The Shannon entropy per barcode.
    """
    read_counts = sorted(read_counts)
    n = len(read_counts)
    total_read_counts = sum(read_counts)
    if total_read_counts == 0:
        return 0
    gini_sum = sum((2 * (i+1) - n - 1) * read_counts[i] for i in range(n))
    gini = gini_sum / (n * total_read_counts)
    return gini

In [ ]:
def shannon_entropy(read_counts):
    """
    Calculate Shannon Entropy for a set of read counts, handling zero read counts appropriately.
    
    Parameters:
    read_counts (list or numpy array): A list or array of read counts per barcode
    
    Returns:
    float: The Shannon entropy per barcode.
    """
    # Convert read counts to a numpy array for easier handling
    read_counts = np.array(read_counts)
    
    # Normalize the read counts to obtain probabilities (p_i)
    total_reads = np.sum(read_counts)
    
    probabilities = read_counts / total_reads
    
    # Calculate Shannon entropy, ignoring zero probabilities
    entropy = 0.0
    for p in probabilities:
        if p > 0:  # Ignore terms where p_i is zero
            entropy -= p * np.log2(p)
    
    return entropy

In [ ]:
def fraction_amplicons_above_threshold(read_counts, threshold):
    """
    This function calculates the fraction of amplicons with read counts greater than or equal to the threshold.
    
    Parameters:
    read_counts (list): A list of read counts for each amplicon.
    threshold (int or float): The threshold value for filtering read counts.
    
    Returns:
    float: Fraction of amplicons with read counts >= threshold.
    """
    
    # Total number of amplicons
    total_amplicons = len(read_counts)
    
    # Avoid division by zero if there are no amplicons
    if total_amplicons == 0:
        return 0
    
    # Count the number of amplicons with reads >= threshold
    amplicons_above_threshold = sum(1 for count in read_counts if count >= threshold)
    
    # Calculate the fraction
    fraction = amplicons_above_threshold / total_amplicons
    
    return fraction

In [ ]:
#Calculate metrics and compile into the target_amplicon_counts.csv
total_reads = count_table.sum(axis=1)
Gini = count_table.apply(lambda row: gini_coefficient(row), axis=1)
evenness = 1 - Gini
entropy = count_table.apply(lambda row: shannon_entropy(row), axis=1)
x1 = count_table.apply(lambda row: fraction_amplicons_above_threshold(row.tolist(), 1), axis=1)
x10 = count_table.apply(lambda row: fraction_amplicons_above_threshold(row.tolist(), 10), axis=1)
x20 = count_table.apply(lambda row: fraction_amplicons_above_threshold(row.tolist(), 20), axis=1)

In [ ]:
read_count_quality_metrics = pd.DataFrame({'Total_reads': total_reads,
                                          'Evenness': evenness,
                                          'Entropy': entropy,
                                          'x1': x1,
                                          'x10': x10,
                                          'x20': x20})
read_count_quality_metrics.to_csv(out_folder + "/read_count_quality_metrics.csv")

## Variant calling & Cell line analysis:
This code is optimized towards 2 cell lines analysis **ONLY**
If you need different cell lines than GM1 and GM2, please contact Caleb @ caleb_thinh_tong@berkeley.edu

In [ ]:
class SingleCell(object):
    # class for storing metadata for each single cell file

    def __init__(self, cell_barcode, out_folder, bam_dir, bam_with_RG_dir, vcf_dir):
        # initialize object by generating filenames

        self.cell_barcode = cell_barcode                    # cell barcode
        self.fastq = f"{out_folder}demuxed_all/cell_{cell_barcode}.fastq.gz"     # fastq file

        self.bam = bam_dir + "cell_" + cell_barcode + '.bam'          # bam file
        self.bai = bam_dir + "cell_" + cell_barcode + '.bai'          # bam file index
        self.bam_with_RG = bam_with_RG_dir + "cell_" + cell_barcode + '.bam'          # bam file
        self.bai_with_RG = bam_with_RG_dir + "cell_" + cell_barcode + '.bai'          # bam file index
        self.vcf = vcf_dir + "cell_" + cell_barcode + '.g.vcf'        # gvcf file

        self.valid = False      # marker for valid cells
        self.alignments = {}    # alignment counts for each interval

    def align_and_index(self, ref_idx, align_cmd = 'bwa-mem2'):

        # align the panel to the bowtie2 human index and generate sorted bam file
        if align_cmd == 'bwa-mem2':
            align_cmd = f"{config['bwamem2_path']} mem -t1" \
                        f' {ref_idx:s} {self.fastq:s}'
        elif align_cmd == 'minimap2':
            pass
        elif align_cmd == 'bowtie2':
            align_cmd = f'bowtie2 -x {ref_idx} --mm --interleaved {self.fastq} --rg-id {self.cell_barcode} ' \
                        f'--rg SM:{self.cell_barcode} --rg PL:ILLUMINA --rg CN:UCSF --quiet'
        else:
            align_cmd = align_cmd

        # filter and sort the output
        align_cmd = align_cmd + f"| {config['samtools_path']} view -b -q 3 -F 4 -F 0X0100 | {config['samtools_path']} sort -o {self.bam:s}"
        
        #print(align_cmd)
        subprocess.call(align_cmd, shell=True)

        # index all bam files using samtools
        index_cmd = f"{config['samtools_path']} index {self.bam} {self.bai}"
        subprocess.call(index_cmd, shell=True)

    def call_variants(self, fasta, interval_file):
        # call variants using gatk

        variants_cmd = f"python3 {config['gatk_path']} HaplotypeCaller " \
                       f'-R %s -I %s -O %s -L %s ' \
                       f'--verbosity ERROR ' \
                       f'--native-pair-hmm-threads 1 ' \
                       f'--standard-min-confidence-threshold-for-calling 0 ' \
                       f'--emit-ref-confidence GVCF ' \
                       f'--max-reads-per-alignment-start 0 ' \
                       f'--max-alternate-alleles 2 ' \
                       f'--minimum-mapping-quality 0' \
                       % (fasta,
                          self.bam_with_RG,
                          self.vcf,
                          interval_file)

        #print(variants_cmd)
        #process = subprocess.Popen(variants_cmd, shell=True)
        subprocess.call(variants_cmd, shell=True)

In [ ]:
file_path = out_folder + "/filtered_barcode_string.txt"
with open(file_path, 'r') as file:
    barcode_recover = file.readlines()
barcode_recover = [line.strip() for line in barcode_recover]

In [ ]:
cells = [
    SingleCell(barcode, out_folder, bam_dir, bam_with_RG_dir, vcf_dir,)
    for barcode in barcode_recover
]

### Genome alignment:

In [ ]:
# ALIGN TO THE REFERENCE GENOME
# limit number of cells to preprocess at a time (based on hardware limitations)
n_preprocess = 80

# create pool of workers and run through all samples
preprocess_pool = ThreadPool(processes=n_preprocess)

for c in cells:
    preprocess_pool.apply_async(SingleCell.align_and_index, args=(c, bwamem2))

preprocess_pool.close()
preprocess_pool.join()

In [ ]:
#Add RG information to each bam file
# Limit number of BAM files to preprocess at a time (based on hardware limitations)
n_preprocess = 80
preprocess_pool = ThreadPool(processes=n_preprocess)

# Define the paths for input and output directories
input_dir = bam_dir
#bam_with_RG_dir = os.path.join(out_folder, "bam_with_RG")

# Ensure the output directory exists
#os.makedirs(bam_with_RG_dir, exist_ok=True)

# Define the Read Group (RG) information
RGID = "1"           # Read group ID
RGLB = "lib1"        # Library
RGPL = "illumina"    # Platform
RGPU = "unit1"       # Platform unit
RGSM = "sample1"     # Sample name

# Function to process a single BAM file
def process_bam(file_name):
    if file_name.endswith('.bam'):
        input_bam = os.path.join(input_dir, file_name)
        output_bam = os.path.join(bam_with_RG_dir, f"{os.path.splitext(file_name)[0]}.bam")
        # Construct the GATK command
        gatk_command = (
            f"python3 {config['gatk_path']} AddOrReplaceReadGroups " \
            f"-I {input_bam} " \
            f"-O {output_bam} " \
            f"--RGID {RGID} " \
            f"--RGLB {RGLB} " \
            f"--RGPL {RGPL} " \
            f"--RGPU {RGPU} " \
            f"--RGSM {RGSM}"
        )
        
        # Run the GATK command
        os.system(gatk_command)
        #print(f"Processed {file_name}")

# List all BAM files in the input directory and process them using ThreadPool
bam_files = [f for f in os.listdir(input_dir) if f.endswith('.bam')]
preprocess_pool.map(process_bam, bam_files)

# Close the ThreadPool
preprocess_pool.close()
preprocess_pool.join()

for file_name in os.listdir(bam_with_RG_dir):
    if file_name.endswith('.bam'):
        input_bam = os.path.join(bam_with_RG_dir, file_name)
        
        # Construct the Samtools index command
        index_command = f"{config['samtools_path']} index {input_bam}"
        
        # Run the Samtools index command
        os.system(index_command)
        #print(f"Indexed {file_name}")

### Variant calling:

In [ ]:
# limit number of cells to call variants at a time (based on hardware limitations)
n_call_variants = 50

# create pool of workers and run through all samples
call_variants_pool = ThreadPool(processes=n_call_variants)

for c in cells:
    call_variants_pool.apply_async(SingleCell.call_variants, args=(c, human_fasta_file, interval_file,))

call_variants_pool.close()
call_variants_pool.join()

In [ ]:
# convert vcf file to csv file for downstream processing
def process_vcf_file(vcf_path, output_folder):
    """
    Processes a single VCF file and saves it as a CSV file in the output folder.
    """
    # Extract the file name
    vcf_file = os.path.basename(vcf_path)

    # Read the header
    with open(vcf_path, 'r') as f:
        for line in f:
            if line.startswith('#CHROM'):
                header = line.strip().lstrip('#').split('\t')
                break

    # Load the VCF data into a DataFrame
    df = pd.read_csv(vcf_path, comment='#', sep='\t', names=header)

    # Split the FORMAT column to create new column names
    format_columns = df['FORMAT'][0].split(':')  # Use the first row's FORMAT string to split into new column names

    # Define a helper function to split and pad/truncate each sample value
    def split_sample(sample, expected_len):
        values = sample.split(':')
        if len(values) > expected_len:
            return values[:expected_len]
        elif len(values) < expected_len:
            return values + [None] * (expected_len - len(values))
        return values

    # Apply the split_sample function to each row of the 'sample1' column
    df_expanded = df['sample1'].apply(lambda x: split_sample(x, len(format_columns))).apply(pd.Series)

    # Assign the new columns from the FORMAT split
    df_expanded.columns = format_columns

    # Combine the original DataFrame with the new expanded columns
    df_final = pd.concat([df.drop(columns=['FORMAT', 'sample1']), df_expanded], axis=1)

    # Create output CSV filename (same as VCF but with .csv extension)
    csv_file_name = os.path.splitext(vcf_file)[0] + '.csv'
    output_csv_path = os.path.join(output_folder, csv_file_name)

    # Save the DataFrame as a CSV
    df_final.to_csv(output_csv_path, index=False)

    #print(f"Processed {vcf_file} and saved as {csv_file_name} in {output_folder}")


def vcf_to_csv(vcf_folder, output_folder, n_preprocess=100):
    """
    Converts VCF files to CSV files using a ThreadPool.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Gather all VCF file paths
    vcf_files = [os.path.join(vcf_folder, f) for f in os.listdir(vcf_folder) if f.endswith('.vcf')]

    # Create ThreadPool with the specified number of threads
    preprocess_pool = ThreadPool(processes=n_preprocess)

    # Submit tasks to the pool
    results = [
        preprocess_pool.apply_async(process_vcf_file, (vcf_path, output_folder))
        for vcf_path in vcf_files
    ]

    # Wait for all tasks to complete
    for result in results:
        result.get()  # Retrieve exceptions if any

    # Close the pool
    preprocess_pool.close()
    preprocess_pool.join()

In [ ]:
%%time
vcf_to_csv(vcf_dir, vcf_csv_dir, n_preprocess=100)

In [ ]:
######### GENOTYPE CALLING
######### ----------------
# Path to the reference file and the directory containing the cell files
reference_file_path = config['truth_GM1_GM2_variant']

# Load the reference file into a DataFrame
reference_df = pd.read_csv(reference_file_path)

# Initialize an empty list to store the data
master_data = []

# Define the mapping for the GT column
gt_mapping = {
    '0/0': 0,
    '0/1': 1,
    '1/0': 1,
    '1/1': 2
}

# Loop through each CSV file in the directory
for file_name in os.listdir(vcf_csv_dir):
    if file_name.endswith('.csv'):  # Only process CSV files
        file_path = os.path.join(vcf_csv_dir, file_name)
        
        # Read the current cell file
        cell_df = pd.read_csv(file_path)
        
        # Merge with reference data on CHROM and POS, keeping only matching rows
        matched_df = pd.merge(cell_df[['CHROM', 'POS', 'GT']], reference_df, on=['CHROM', 'POS'], how='right')
        
        # If there are no matches (NaNs in GT), set GT to 3
        matched_df['GT'] = matched_df['GT'].fillna(3)
        
        # Map the GT column using the defined mapping
        matched_df['GT'] = matched_df['GT'].map(gt_mapping).fillna(3).astype(int)
        
        # Add a column to indicate the file source
        matched_df['File name'] = file_name
        
        # Drop unwanted columns ("CELL_LINES" and "CALL")
        matched_df = matched_df.drop(columns=['CELL_LINES', 'CALL'], errors='ignore')
        
        # Modify the "File name" column to strip off the ".g.csv" part
        matched_df['File name'] = matched_df['File name'].str.replace('.g.csv$', '', regex=True)
        
        # Append the processed data to the master list
        master_data.append(matched_df)

# Combine all individual DataFrames into one master DataFrame
master_df = pd.concat(master_data, ignore_index=True)
master_df['Variant'] = master_df['CHROM'].astype(str) + ":" + master_df['POS'].astype(str)
master_df = master_df.drop(columns = ['CHROM', 'POS'])
master_df = master_df.pivot_table(index='File name', columns='Variant', values='GT', aggfunc='first')

# Optionally, save the master DataFrame to a CSV file
master_df.to_csv(out_folder + '/genotype_compiling.csv')

### Cell-line related metrics: (Jaccard only for now)

In [ ]:
def jaccard_index_with_all(df, row_idx):
    """
    Calculate the Jaccard Index between one row and all rows in a dataframe.
    
    Parameters:
        df (pd.DataFrame): The dataframe.
        row_idx (int): Index of the row to compare against all rows.

    Returns:
        pd.Series: Jaccard indices for each row in the dataframe.
    """
    # Select the target row
    target_row = df.loc[row_idx]
    
    # Calculate Jaccard Index for each row
    def calculate_jaccard(row):
        # Intersection: Values that match between the two rows
        intersection = (target_row == row).sum()
        # Union: Total length minus the intersection
        union = 2*len(row) - intersection
        # Jaccard Index (skip division by zero check for now)
        return intersection / union

    # Apply the Jaccard Index calculation to all rows
    jaccard_indices = df.apply(calculate_jaccard, axis=1)
    
    return jaccard_indices

In [ ]:
reference_df['CHROM:POS'] = reference_df['CHROM'] + ':' + reference_df['POS'].astype(str)
reference_df = reference_df.drop(['CHROM', 'POS'], axis=1)
pivot_df = reference_df.pivot(index='CELL_LINES', columns='CHROM:POS', values='CALL')
pivot_df = pivot_df.fillna(0)
ref_df = pivot_df
ref_df = pd.DataFrame(ref_df)

In [ ]:
snp_df_combine = pd.concat([ref_df, master_df])
snp_df_combine_drop = snp_df_combine.drop(['chr4:54736599', 'chr7:148808972'], axis=1)
snp_df_combine
cell_line_1_jaccard = jaccard_index_with_all(snp_df_combine_drop, 'GM12878')
cell_line_2_jaccard = jaccard_index_with_all(snp_df_combine_drop, 'GM24385')
jaccard_df = pd.DataFrame({'Jaccard_GM12878': cell_line_1_jaccard,
                          'Jaccard_GM24385': cell_line_2_jaccard})
jaccard_df.to_csv(out_folder + '/Jaccard_metrics.csv')